In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_attention_encoder_functions import (
    seqsignet_attention_encoder_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# SeqSigNet with Attention Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [8]:
split_ids = torch.tensor(df_rumours['timeline_id'].astype(int))

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]#, (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "transformer_encoder_layers": 1,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [11]:
shift = 3
window_size = 5
n = 3

## Random Projections

In [12]:
(
    seqsignet_attention_encoder_grp_11,
    best_seqsignet_attention_encoder_grp_11,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_3_kfold_best_model.csv


In [13]:
seqsignet_attention_encoder_grp_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1043650/3905548123.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_encoder_grp_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.683442   
                                                                                      0.0005         0.698664   
                                                                                      0.0010         0.702827   
                                                                         0.2          0.0001         0.688723   
                                                                                      0.0005         0.693880   
                                                                                      0.0010         0.704629   
                                                          (512, 512)     0.1          0.0001         0.686735   
                                                                                      0.0005         0.696676   
                                                                                      0.0010         0.694501   
                                                                         0.2          0.0001         0.694191   
                                                                                      0.0005         0.684188   
                                                                                      0.0010         0.695433   
           12              3         10        1          (256, 256)     0.1          0.0001         0.675738   
                                                                                      0.0005         0.699907   
                                                                                      0.0010         0.697732   
                                                                         0.2          0.0001         0.676483   
                                                                                      0.0005         0.700777   
                                                                                      0.0010         0.699845   
                                                          (512, 512)     0.1          0.0001         0.672693   
                                                                                      0.0005         0.686424   
                                                                                      0.0010         0.705064   
                                                                         0.2          0.0001         0.673004   
                                                                                      0.0005         0.693259   
                                                                                      0.0010         0.695061   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.652011   
                                                                                      0.0005         0.668237   
                                                                                      0.0010         0.667858   
                                                                         0.2          0.0001         0.606914   
                                                                                      0.0005         0.662904   
                                                                                      0.0010         0.667058   
                                                          (512, 512)     0.1          0.0001         0.649685   
                                                                                      0.0005         0.658761   
                                

In [14]:
best_seqsignet_attention_encoder_grp_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.671202,0.658492,"[0.724375, 0.5926096997690532]",0.659525,"[0.7960164835164835, 0.5230330207908683]",0.674051,"[0.6645642201834863, 0.6835375599360682]",None,0.756301,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.675116,0.662279,"[0.7281235376696303, 0.5964343598055106]",0.662966,"[0.7984946972288745, 0.5274365274365275]",0.677676,"[0.669151376146789, 0.6862013851891315]",None,0.766141,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.671761,0.658111,"[0.7264253534255087, 0.5897973445143256]",0.658409,"[0.7928111224143778, 0.5240066225165563]",0.672389,"[0.6702981651376146, 0.6744805540756527]",None,0.766680,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_seqsignet_attention_encoder_grp_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001


in path: 0.658

in input: 0.6430702845314854

both: 0.6596275491973381

In [16]:
best_seqsignet_attention_encoder_grp_11["f1"].mean()

0.6596275491973381

In [17]:
best_seqsignet_attention_encoder_grp_11["precision"].mean()

0.6602997456506147

In [18]:
best_seqsignet_attention_encoder_grp_11["recall"].mean()

0.6747055434447904

In [19]:
np.stack(best_seqsignet_attention_encoder_grp_11["f1_scores"]).mean(axis=0)

array([0.72630796, 0.59294713])

In [20]:
np.stack(best_seqsignet_attention_encoder_grp_11["precision_scores"]).mean(axis=0)

array([0.7957741 , 0.52482539])

In [21]:
np.stack(best_seqsignet_attention_encoder_grp_11["recall_scores"]).mean(axis=0)

array([0.66800459, 0.6814065 ])

# history_length=20

In [22]:
shift = 3
window_size = 5
n = 6

## Random Projections

In [23]:
(
    seqsignet_attention_encoder_grp_20,
    best_seqsignet_attention_encoder_grp_20,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_6_kfold_best_model.csv


In [24]:
seqsignet_attention_encoder_grp_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1043650/1706734633.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_encoder_grp_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.674806   
                                                                                      0.0005         0.694129   
                                                                                      0.0010         0.700217   
                                                                         0.2          0.0001         0.685989   
                                                                                      0.0005         0.700777   
                                                                                      0.0010         0.699472   
                                                          (512, 512)     0.1          0.0001         0.668282   
                                                                                      0.0005         0.695744   
                                                                                      0.0010         0.691892   
                                                                         0.2          0.0001         0.698354   
                                                                                      0.0005         0.688040   
                                                                                      0.0010         0.695123   
           12              3         10        1          (256, 256)     0.1          0.0001         0.671513   
                                                                                      0.0005         0.685057   
                                                                                      0.0010         0.691146   
                                                                         0.2          0.0001         0.679652   
                                                                                      0.0005         0.689034   
                                                                                      0.0010         0.699161   
                                                          (512, 512)     0.1          0.0001         0.683069   
                                                                                      0.0005         0.697484   
                                                                                      0.0010         0.703697   
                                                                         0.2          0.0001         0.678099   
                                                                                      0.0005         0.696365   
                                                                                      0.0010         0.695061   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.626625   
                                                                                      0.0005         0.665069   
                                                                                      0.0010         0.667265   
                                                                         0.2          0.0001         0.586221   
                                                                                      0.0005         0.657054   
                                                                                      0.0010         0.668886   
                                                          (512, 512)     0.1          0.0001         0.646247   
                                                                                      0.0005         0.667666   
                                

In [25]:
best_seqsignet_attention_encoder_grp_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.660578,0.642640,"[0.7227044312471448, 0.5625750660581311]",0.641477,"[0.7707047742773627, 0.5122484689413823]",0.652100,"[0.6803325688073395, 0.623867874267448]",None,0.845667,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.684436,0.665452,"[0.7451452656932109, 0.5857597259603622]",0.663049,"[0.7844690966719493, 0.5416289592760181]",0.673648,"[0.7095756880733946, 0.6377197655833777]",None,0.846206,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.669525,0.652121,"[0.7299314546839301, 0.5743097238895558]",0.650704,"[0.7786805329866753, 0.5227272727272727]",0.662057,"[0.6869266055045872, 0.637187000532765]",None,0.836636,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [26]:
best_seqsignet_attention_encoder_grp_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [27]:
best_seqsignet_attention_encoder_grp_20["f1"].mean()

0.6534042779220558

In [28]:
best_seqsignet_attention_encoder_grp_20["precision"].mean()

0.6517431841467768

In [29]:
best_seqsignet_attention_encoder_grp_20["recall"].mean()

0.6626015837948187

In [30]:
np.stack(best_seqsignet_attention_encoder_grp_20["f1_scores"]).mean(axis=0)

array([0.73259372, 0.57421484])

In [31]:
np.stack(best_seqsignet_attention_encoder_grp_20["precision_scores"]).mean(axis=0)

array([0.77795147, 0.5255349 ])

In [32]:
np.stack(best_seqsignet_attention_encoder_grp_20["recall_scores"]).mean(axis=0)

array([0.69227829, 0.63292488])

# history_length=35

In [33]:
shift = 3
window_size = 5
n = 11

## Random Projections

In [34]:
(
    seqsignet_attention_encoder_grp_35,
    best_seqsignet_attention_encoder_grp_35,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_11_kfold_best_model.csv


In [35]:
seqsignet_attention_encoder_grp_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1043650/1464355546.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_encoder_grp_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.669960   
                                                                                      0.0005         0.699845   
                                                                                      0.0010         0.693321   
                                                                         0.2          0.0001         0.686238   
                                                                                      0.0005         0.696490   
                                                                                      0.0010         0.689655   
                                                          (512, 512)     0.1          0.0001         0.680957   
                                                                                      0.0005         0.697173   
                                                                                      0.0010         0.697297   
                                                                         0.2          0.0001         0.689345   
                                                                                      0.0005         0.693818   
                                                                                      0.0010         0.700217   
           12              3         10        1          (256, 256)     0.1          0.0001         0.685306   
                                                                                      0.0005         0.693197   
                                                                                      0.0010         0.701895   
                                                                         0.2          0.0001         0.687853   
                                                                                      0.0005         0.698105   
                                                                                      0.0010         0.698478   
                                                          (512, 512)     0.1          0.0001         0.684747   
                                                                                      0.0005         0.685555   
                                                                                      0.0010         0.690587   
                                                                         0.2          0.0001         0.673190   
                                                                                      0.0005         0.696241   
                                                                                      0.0010         0.698602   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.624108   
                                                                                      0.0005         0.669399   
                                                                                      0.0010         0.666014   
                                                                         0.2          0.0001         0.635590   
                                                                                      0.0005         0.663949   
                                                                                      0.0010         0.660947   
                                                          (512, 512)     0.1          0.0001         0.652143   
                                                                                      0.0005         0.665723   
                                

In [36]:
best_seqsignet_attention_encoder_grp_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.665797,0.640161,"[0.7362071502133294, 0.5441139079583016]",0.638274,"[0.7561196736174071, 0.5204280155642024]",0.643688,"[0.7173165137614679, 0.5700586041555674]",None,0.869255,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.703448,0.675674,"[0.7705839942321557, 0.580764163372859]",0.674862,"[0.7751668117203365, 0.5745568300312826]",0.676581,"[0.7660550458715596, 0.5871070857751731]",None,0.883677,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.686673,0.655832,"[0.7588581265241716, 0.5528065974993349]",0.655738,"[0.7594028136663795, 0.5520722635494155]",0.655929,"[0.7583142201834863, 0.5535428875865743]",None,0.871681,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [37]:
best_seqsignet_attention_encoder_grp_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [38]:
best_seqsignet_attention_encoder_grp_35["f1"].mean()

0.6572223233000253

In [39]:
best_seqsignet_attention_encoder_grp_35["precision"].mean()

0.6562910680248373

In [40]:
best_seqsignet_attention_encoder_grp_35["recall"].mean()

0.6587323928889715

In [41]:
np.stack(best_seqsignet_attention_encoder_grp_35["f1_scores"]).mean(axis=0)

array([0.75521642, 0.55922822])

In [42]:
np.stack(best_seqsignet_attention_encoder_grp_35["precision_scores"]).mean(axis=0)

array([0.7635631 , 0.54901904])

In [43]:
np.stack(best_seqsignet_attention_encoder_grp_35["recall_scores"]).mean(axis=0)

array([0.74722859, 0.57023619])

# history_length=80

In [11]:
shift = 3
window_size = 5
n = 26

## UMAP

## Random Projections

In [12]:
(
    seqsignet_attention_encoder_grp_80,
    best_seqsignet_attention_encoder_grp_80,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_grp_focal_2_3_5_26_kfold_best_model.csv


In [13]:
seqsignet_attention_encoder_grp_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2158554/4204647423.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_encoder_grp_80.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.677664   
                                                                                      0.0005         0.704442   
                                                                                      0.0010         0.700093   
                                                                         0.2          0.0001         0.685306   
                                                                                      0.0005         0.700342   
                                                                                      0.0010         0.699658   
                                                          (512, 512)     0.1          0.0001         0.681081   
                                                                                      0.0005         0.699223   
                                                                                      0.0010         0.698291   
                                                                         0.2          0.0001         0.688040   
                                                                                      0.0005         0.691954   
                                                                                      0.0010         0.696365   
           12              3         10        1          (256, 256)     0.1          0.0001         0.689531   
                                                                                      0.0005         0.692203   
                                                                                      0.0010         0.699534   
                                                                         0.2          0.0001         0.696862   
                                                                                      0.0005         0.703262   
                                                                                      0.0010         0.703945   
                                                          (512, 512)     0.1          0.0001         0.692389   
                                                                                      0.0005         0.695993   
                                                                                      0.0010         0.693507   
                                                                         0.2          0.0001         0.680708   
                                                                                      0.0005         0.699596   
                                                                                      0.0010         0.695433   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.647965   
                                                                                      0.0005         0.668731   
                                                                                      0.0010         0.671142   
                                                                         0.2          0.0001         0.653579   
                                                                                      0.0005         0.666742   
                                                                                      0.0010         0.667484   
                                                          (512, 512)     0.1          0.0001         0.654769   
                                                                                      0.0005         0.668661   
                                

In [14]:
best_seqsignet_attention_encoder_grp_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.682013,0.652768,"[0.7535394394683618, 0.5519957983193278]",0.651871,"[0.7594641817122889, 0.5442775763852926]",0.653821,"[0.7477064220183486, 0.5599360681939265]",None,0.884081,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.680149,0.650401,"[0.7523809523809523, 0.548421052631579]",0.649635,"[0.7574084834398606, 0.5418616744669786]",0.651280,"[0.7474197247706422, 0.5551411827384124]",None,0.885429,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.670829,0.640725,"[0.7447239086441169, 0.5367261280167891]",0.639851,"[0.7510204081632653, 0.5286821705426357]",0.641775,"[0.7385321100917431, 0.5450186467767715]",None,0.872490,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_seqsignet_attention_encoder_grp_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,8,4,6,1,"(256, 256)",0.1,0.0001
0,15,8,4,6,1,"(256, 256)",0.1,0.0001
0,15,8,4,6,1,"(256, 256)",0.1,0.0001


In [16]:
best_seqsignet_attention_encoder_grp_80["f1"].mean()

0.6479645465768544

In [17]:
best_seqsignet_attention_encoder_grp_80["precision"].mean()

0.6471190824517202

In [18]:
best_seqsignet_attention_encoder_grp_80["recall"].mean()

0.6489590257649741

In [19]:
np.stack(best_seqsignet_attention_encoder_grp_80["f1_scores"]).mean(axis=0)

array([0.75021477, 0.54571433])

In [20]:
np.stack(best_seqsignet_attention_encoder_grp_80["precision_scores"]).mean(axis=0)

array([0.75596436, 0.53827381])

In [21]:
np.stack(best_seqsignet_attention_encoder_grp_80["recall_scores"]).mean(axis=0)

array([0.74455275, 0.5533653 ])